In [ ]:
workdir="BayesianDrape/"
scriptfilename=workdir+"mpiscript.py"

In [ ]:
%%writefile $scriptfilename
# parallelization as per https://emcee.readthedocs.io/en/stable/tutorials/parallel/

bdscript="BayesianDrape/BayesianDrape.py"
datadir="BayesianDrape/data"

import geopandas as gp
import pandas as pd
import numpy as np
from scipy.stats import linregress
import subprocess
import os,sys,socket
import emcee
import statsmodels.api as sm
import pylab
from scipy.stats import norm
import seaborn as sn
import corner
from scipy import optimize,linalg
import matplotlib.pyplot as plt
from schwimmbad import MPIPool 

def shell(cmd):
    process = subprocess.Popen([sys.executable]+cmd.split(" "),
                     stdout=subprocess.PIPE, 
                     stderr=subprocess.PIPE,text=True)
    stdout, stderr = process.communicate()
    if not process.returncode==0:
        try:
            print(stdout)
            print(stderr)
        except:
            print("failed to decode stdout/stderr")
        return False
    return True


def test_drape(spatial_mismatch_prior_scale,slope_prior_scale,slope_continuity_prior_scale,pitch_angle_prior_scale):
    outbase=f"{socket.gethostname()}-{os.getpid()}"
    bayes_output_filename = outbase+"-bdrape.shp"
    res=shell(f"{bdscript} --TERRAIN-INPUT={datadir}/all_os50_terrain.tif --POLYLINE-INPUT={datadir}/biggertest.shp "
          f"--OUTPUT={bayes_output_filename} --SPATIAL-MISMATCH-PRIOR-STD={spatial_mismatch_prior_scale} "
          f"--SLOPE-CONTINUITY-PRIOR-SCALE={slope_continuity_prior_scale} --SLOPE-PRIOR-SCALE={slope_prior_scale} "
          f"--USE-PITCH-ANGLE-PRIOR --PITCH-ANGLE-PRIOR-SCALE={pitch_angle_prior_scale} "
          f"--MAXITER=10000 --DECOUPLE-FIELD=bridge")
    if res:
        return bayes_output_filename
    else:
        return False
    
def markov_speed_factor(slope):
    '''https://link.springer.com/article/10.1007/s11116-019-10021-x#Tab5'''
    return (slope>0)*np.exp(-9.044*slope)+(slope<=0)*1

from itertools import tee
def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

def cycletime(slope,length):
    # bodge based on linear model for now:
    # 0% slope = factor of 1
    # 5% slope = factor of 2.2 
    # slopefac = slope*100/5*1.2+1
    slopefac = 1./markov_speed_factor(slope)
    return slopefac*length

# this is equiv to ct = ec*100/5*1.2 + len when previously we were comparing on ec
# i.e. comparing on 24*ec+len which will lessen importance of ec on long links
# is this the desired result? i wanted to study outliers more; i guess this studies outliers of grade?

def resid_stats(df,x,y,printstats):
    lr = linregress(df[x],df[y])
    df["resid"] = df[y]-(lr.intercept+lr.slope*df[x])
    scale = df.resid.std()
    loglik = norm(loc=0,scale=scale).logpdf(df.resid).sum()

    if printstats:
        df.plot.scatter(x,y)
        df.plot.scatter("length","resid")
        print (f"{y} vs {x}\n{lr.rvalue=}\n{df.resid.mean()=}\n{abs(df.resid).sum()/df.length.sum()=}\n{loglik=}\n")
        
    return loglik
        
def elev_change(geom):
    ec = 0
    for (_,_,z1),(_,_,z2) in pairwise(list(geom.coords)):
        ec += abs(z2-z1)
    return ec
    
def post_log_likelihood(smps,sps=90,continrat=np.inf,paps=np.inf,printstats=False,compare="ELEVCHANGE"):
    scps = smps*continrat
    #slope = np.arctan(sloperat*smp)*180/np.pi did i have this before to remove interactions?
    draped_net_filename = test_drape(smps,sps,scps,paps)
    if not draped_net_filename:
        return -np.inf
    
    draped_net = gp.read_file(draped_net_filename)

    draped_net["draped_elev_change"]=[elev_change(geom) for geom in draped_net.geometry]
    draped_net["length"]=draped_net.SHAPE_Leng
    
    def height(s):
        assert s[-1]=="m"
        return float(s[0:-1])
    draped_net["os_elev_change"] = draped_net.apply(lambda row: height(row["elevationG"])+height(row["elevatio_1"]),
                                                    axis=1)
    
    if printstats or compare=="CYCLETIME":
        draped_net["os_slope"] = draped_net.os_elev_change/draped_net.length
        draped_net["draped_slope"] = draped_net.draped_elev_change/draped_net.length
        draped_net["os_cycletime"] = cycletime(draped_net.os_slope,draped_net.length)
        draped_net["draped_cycletime"] = cycletime(draped_net.draped_slope,draped_net.length)
    
    if printstats:
        resid_stats(draped_net,"os_elev_change","draped_elev_change",True)
        resid_stats(draped_net,"os_slope","draped_slope",True)
        resid_stats(draped_net,"os_cycletime","draped_cycletime",True)
        
    if compare=="ELEVCHANGE":
        return resid_stats(draped_net,"os_elev_change","draped_elev_change",False)
    elif compare=="CYCLETIME":
        resid_stats(draped_net,"os_cycletime","draped_cycletime",False)
    else:
        assert False
     
def run_mcmc(ll_func,param_labels,start_guess_min,start_guess_max,n_walk,n_steps,n_burn,name):

    with MPIPool() as pool:
        if not pool.is_master():
            pool.wait()
            sys.exit(0)
        
        n_dim = len(param_labels)
        assert len(start_guess_min)==n_dim
        assert len(start_guess_max)==n_dim
        starting_guesses = [start_guess_min + (start_guess_max-start_guess_min)*np.random.random(n_dim) 
                        for _ in range(n_walk)]

        sampler = emcee.EnsembleSampler(n_walk, n_dim, ll_func, pool=pool)

        # Run sampler
        print ("Running MCMC")
        pos, prob, state = sampler.run_mcmc(starting_guesses, n_steps, progress=True)

        # Print some stats. based on run properties
        print ('Average acceptance fraction: ', np.mean(sampler.acceptance_fraction))
        #print 'Autocorrelation time: ', sampler.acor

        # Get results
        # Plot traces, including burn-in
        fig, axes = plt.subplots(nrows=n_dim, ncols=1, figsize=(10, 10))    
        for idx, title in enumerate(param_labels):        
            axes[idx].plot(sampler.chain[:,:,idx].T, '-', color='k', alpha=0.3)
            axes[idx].set_title(title, fontsize=20) 
        plt.subplots_adjust(hspace=0.5)
        plt.savefig(f"{name}-traces.png")

        # Discard burn-in
        samples = sampler.chain[:, n_burn:, :].reshape((-1, n_dim))

        # Triangle plot
        tri = corner.corner(samples,
                            labels=param_labels,
                            quantiles=[0.025, 0.5, 0.975],
                            show_titles=True, 
                            title_args={'fontsize': 24},
                            label_kwargs={'fontsize': 20})
        tri.savefig(f"{name}-corner.png")

        # Get the median param estimates
        medians = np.median(samples, axis=0)

        print(param_labels)
        print(medians)

def log_likelihood(mcmc_params):
    smp,continrat = mcmc_params
    if 0<=smp<=200 and 0<=continrat<=0.1 :
        return post_log_likelihood(smp,continrat=continrat,compare="CYCLETIME")
    else:
        return -np.inf
    
param_labels = ["mismatch","continrat"]
param_guess_min = np.array([1,0.001])
param_guess_max = np.array([200,0.1])

#16 300 150
n_walk = 4    # Number of "walkers"/chains
n_steps = 20  # Number of steps per chain
n_burn = 0   # Length of burn-in to discard

run_mcmc(log_likelihood,param_labels,param_guess_min,param_guess_max,n_walk,n_steps,n_burn,"sm-sc")

In [ ]:
#ntasks = !sstat -o NTasks $$SLURM_JOBID | tail -n 1
#print (f"running with {ntasks} tasks")
#!mpiexec -n $ntasks python -u $scriptfilename

# seems to launch the correct num tasks by default:
!mpiexec python -u $scriptfilename